In [91]:
import requests
import json
import time
import pickle
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon
# import censusdata
from sklearn.metrics.pairwise import haversine_distances
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
import math
from bs4 import BeautifulSoup as BS
import ast
from itertools import combinations
from collections import Counter
#from scipy.cluster.hierarchy 
from pyproj import Geod
from shapely.geometry import Point, LineString

%matplotlib inline

In [92]:
hidden = pd.read_pickle('../data/API_key.pkl')
api_key = hidden.loc[0][1]

In [93]:
davidson_pred_black = gpd.read_file('../data/davidson_race_by_tract/davidson_tracts_pred_black.geojson')
davidson_pred_black.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   geoid       25 non-null     object  
 1   name        25 non-null     object  
 2   total_pop   25 non-null     float64 
 3   total_pop_  25 non-null     float64 
 4   white_alon  25 non-null     float64 
 5   white_al_1  25 non-null     float64 
 6   black_afri  25 non-null     float64 
 7   black_af_1  25 non-null     float64 
 8   nativeamer  25 non-null     float64 
 9   nativeam_1  25 non-null     float64 
 10  asian_alon  25 non-null     float64 
 11  asian_al_1  25 non-null     float64 
 12  hawaiian_p  25 non-null     float64 
 13  hawaiian_1  25 non-null     float64 
 14  other_alon  25 non-null     float64 
 15  other_al_1  25 non-null     float64 
 16  two_or_mor  25 non-null     float64 
 17  two_or_m_1  25 non-null     float64 
 18  two_includ  25 non-null     float64 
 19  tw

In [125]:
location_10105 = '36.24885, -86.85388' # lat, lng as a string
radius_10105 = '4534' 
all_types = ['gas_station',
         'grocery_or_supermarket',
         'hospital',
         'clothing_store',
        'restaurant'] 

In [65]:
# Modified from https://python.gotrained.com/google-places-api-extracting-location-data-reviews/

def search_places_by_coordinate(location, radius, types, api_key, sleep_sec = 2):
    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    places = []
    params = {
        'location': location,
        'radius': radius,
        'types': types,
        'key': api_key
    }
    res = requests.get(endpoint_url, params = params)
    results =  res.json()
    places.extend(results['results'])
    time.sleep(sleep_sec)
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.get(endpoint_url, params = params)
        results = res.json()
        places.extend(results['results'])
        time.sleep(sleep_sec)
    return places

In [126]:
ct10105 = search_places_by_coordinate(location_10105, radius_10105, all_types, api_key)

In [127]:
len(ct10105)

7

In [128]:
print(ct10105)

[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 36.2520693, 'lng': -86.8297597}, 'viewport': {'northeast': {'lat': 36.2534926302915, 'lng': -86.8284251697085}, 'southwest': {'lat': 36.2507946697085, 'lng': -86.8311231302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/gas_station-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/gas_pinlet', 'name': 'Shell', 'photos': [{'height': 4128, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117265797567492229445">Rooster Davis</a>'], 'photo_reference': 'Aap_uED-U9l0XSClaB5NUn3A3dR0aBR67y3bngsrfnsW3TwhxjyjSJUd8ASRaRDTGw0YQK-32PqVGyIZ0SPqMgLWKSVsc5OGAuYDyIp31OSG5jKDfj8s9nD_4YJ3nrzPfdsdYWc53nQgLuNQUWuyt-JcbMmA3p4YXdEkUu5cRTycqPuAo2Pw', 'width': 3096}], 'place_id': 'ChIJBQHTXnBeZIgR4BxNPRfisdI', 'plus_code': {'compound_code': '752C+R3 Nashville, TN, USA', 'global_code': '868M752C+R3'}, 'price_level': 2, 'rati

In [129]:
ct10105[0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 36.2520693, 'lng': -86.8297597},
  'viewport': {'northeast': {'lat': 36.2534926302915,
    'lng': -86.8284251697085},
   'southwest': {'lat': 36.2507946697085, 'lng': -86.8311231302915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/gas_station-71.png',
 'icon_background_color': '#909CE1',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/gas_pinlet',
 'name': 'Shell',
 'photos': [{'height': 4128,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117265797567492229445">Rooster Davis</a>'],
   'photo_reference': 'Aap_uED-U9l0XSClaB5NUn3A3dR0aBR67y3bngsrfnsW3TwhxjyjSJUd8ASRaRDTGw0YQK-32PqVGyIZ0SPqMgLWKSVsc5OGAuYDyIp31OSG5jKDfj8s9nD_4YJ3nrzPfdsdYWc53nQgLuNQUWuyt-JcbMmA3p4YXdEkUu5cRTycqPuAo2Pw',
   'width': 3096}],
 'place_id': 'ChIJBQHTXnBeZIgR4BxNPRfisdI',
 'plus_code': {'compound_code': '752C+R3 Nashville, TN, USA',
  'global_code': '868M752C+R3

In [130]:
print(ct10105[0]['geometry']['location']['lat'])
print(ct10105[0]['geometry']['location']['lng'])
print(ct10105[0]['name'])
print(ct10105[0]['types'][0])

36.2520693
-86.8297597
Shell
gas_station


In [132]:
latitude10105 = []
longitude10105 = []
name10105 = []
type10105 = []

for i in ct10105:
    latitude10105.append(i['geometry']['location']['lat'])
    longitude10105.append(i['geometry']['location']['lng'])
    name10105.append(i['name'])
    type10105.append(i['types'][0])

In [137]:
name10105

['Shell',
 "Cenex KCK's",
 'Kroger Fuel Center',
 "Brenda's Corner Market",
 'Kwik Sak',
 'Citgo',
 'Friendly Corner Market']

In [134]:
df10105 = pd.DataFrame(list(zip(name10105, type10105, latitude10105, longitude10105)),
               columns =['name', 'type', 'latitude', 'longitude'])

In [136]:
df10105['tract'] = 'census_tract_10105'
df10105

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.252069,-86.829760,census_tract_10105
1,Cenex KCK's,gas_station,36.279847,-86.873683,census_tract_10105
2,Kroger Fuel Center,gas_station,36.210674,-86.838564,census_tract_10105
3,Brenda's Corner Market,convenience_store,36.280015,-86.873743,census_tract_10105
4,Kwik Sak,gas_station,36.218140,-86.836695,census_tract_10105
5,Citgo,gas_station,36.279806,-86.873688,census_tract_10105
6,Friendly Corner Market,convenience_store,36.279847,-86.873683,census_tract_10105


In [88]:
test_map = folium.Map([36.24885, -86.85388])
for row_index, row_values in test_df.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    tool = [str(row_values['name']), str(row_values['type'])]
    
    marker = folium.Marker(
        location = loc, 
        tooltip = tool) 

    marker.add_to(test_map)
test_map

In [100]:
location_10106 = '36.25371, -86.80361' # lat, lng as a string
radius_10106 = '4000'

In [101]:
ct10106 = search_places_by_coordinate(location_10106, radius_10106, all_types, api_key)
len(ct10106)

8

In [113]:
test_dict = {location_10105:radius_10105, location_10106:radius_10106}

In [119]:
test_name = []
test_type = []

for key, value in test_dict.items():
    results_10105_10106 =  search_places_by_coordinate(key, value, all_types, api_key)
    for i in results_10105_10106:
        test_name.append(i['name'])
        test_type.append(i['types'][0])


In [123]:
print(test_type)

['gas_station', 'gas_station', 'gas_station', 'convenience_store', 'gas_station', 'convenience_store', 'gas_station', 'convenience_store', 'gas_station', 'gas_station', 'convenience_store', 'gas_station', 'convenience_store', 'convenience_store']


In [117]:
len(results_10105_10106)

8

# Now that I've created the DataFrame for the first census tract, I will do the same for the others